# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [2]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [3]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [4]:
# confirm that our new database was created
print(mongo.list_database_names())


['admin', 'classDB', 'config', 'gardenDB', 'local', 'met', 'petsitly_marketing', 'travel_db']


In [5]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [6]:
# review the collections in our new database
print(db.list_collection_names())

[]


In [ ]:
# review a document in the establishments collection
    

In [10]:
# assign the collection to a variable
establishments = db['establishments']                   

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [8]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName": "The Food Place",
    "BusinessType": "Restaurant",
    "BusinessTypeID": 1,
    "AddressLine1": "123 Food St",
    "AddressLine2": "",
    "AddressLine3": "",
    "PostCode": "AB12 3CD",
    "Phone": "01234 567890",
    "RatingValue": 5,
    "RatingKey": 1,
    "RatingDate": "2023-10-01",
    "LocalAuthorityCode": "E12345678",
    "LocalAuthorityName": "Food City Council",
    "LocalAuthorityWebSite": "http://www.foodcity.gov.uk",
}

In [11]:
# Insert the new restaurant into the collection
result = establishments.insert_one(new_restaurant)

In [12]:
# Check that the new restaurant was inserted
print("Inserted restaurant with ID:", result.inserted_id)

Inserted restaurant with ID: 68302438d2eb29153594b0a7


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [13]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
pipeline = [
    {
        "$match": {
            "BusinessType": "Restaurant/Cafe/Canteen"
        }
    },
    {
        "$project": {
            "_id": 0,
            "BusinessTypeID": 1,
            "BusinessType": 1
        }
    }
]

3. Update the new restaurant with the `BusinessTypeID` you found.

In [15]:
# Update the new restaurant with the correct BusinessTypeID
result = establishments.update_one(
    {"BusinessName": "The Food Place"},
    {
        "$set": {
            "BusinessTypeID": 1
        }
    }
)

In [16]:
# Confirm that the new restaurant was updated
print("Updated restaurant with ID:", result.upserted_id if result.upserted_id else "No new restaurant was created")
pprint(establishments.find_one({"BusinessName": "The Food Place"}))


Updated restaurant with ID: No new restaurant was created
{'AddressLine1': '123 Food St',
 'AddressLine2': '',
 'AddressLine3': '',
 'BusinessName': 'The Food Place',
 'BusinessType': 'Restaurant',
 'BusinessTypeID': 1,
 'LocalAuthorityCode': 'E12345678',
 'LocalAuthorityName': 'Food City Council',
 'LocalAuthorityWebSite': 'http://www.foodcity.gov.uk',
 'Phone': '01234 567890',
 'PostCode': 'AB12 3CD',
 'RatingDate': '2023-10-01',
 'RatingKey': 1,
 'RatingValue': 5,
 '_id': ObjectId('68302438d2eb29153594b0a7')}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [17]:
# Find how many documents have LocalAuthorityName as "Dover"
pipeline = [
    {
        "$match": {
            "LocalAuthorityName": "Dover"
        }
    },
    {
        "$count": "number_of_documents"
    }
]

In [18]:
# Delete all documents where LocalAuthorityName is "Dover"
result = establishments.delete_many({"LocalAuthorityName": "Dover"})

In [19]:
# Check if any remaining documents include Dover
remaining_documents = establishments.find({"LocalAuthorityName": "Dover"})


In [20]:
# Check that other documents remain with 'find_one'
remaining_document = establishments.find_one({"LocalAuthorityName": "Dover"})
if remaining_document:
    print("Remaining document with LocalAuthorityName 'Dover':", remaining_document)


5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [21]:
# Change the data type from String to Decimal for longitude and latitude
from bson.decimal128 import Decimal128
from bson import Decimal128
from pymongo import UpdateOne

Use `update_many` to convert `RatingValue` to integer numbers.

In [22]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

In [23]:
# Change the data type from String to Integer for RatingValue
establishments.update_many(
    {"RatingValue": {"$type": "string"}},
    [
        {
            "$set": {
                "RatingValue": {
                    "$convert": {
                        "input": "$RatingValue",
                        "to": "int",
                        "onError": None,
                        "onNull": None
                    }
                }
            }
        }
    ]
)

UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

In [24]:
# Check that the coordinates and rating value are now numbers
establishments.update_many(
    {"geocode.longitude": {"$type": "string"}},
    [
        {
            "$set": {
                "geocode.longitude": {
                    "$convert": {
                        "input": "$geocode.longitude",
                        "to": "double",
                        "onError": None,
                        "onNull": None
                    }
                }
            }
        }
    ]
)

UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)